## Equivalent MIP aimed by the FAST algorithm

In [ ]:
# Import packages
using JuMP
using Cbc, HiGHS, Clp
using SpineInterface

In [ ]:
# activate SpineDB
inputDB = "sqlite:///$(@__DIR__)\\.spinetoolbox\\items\\input_db\\Input DB.sqlite"
# "sqlite:///$(pwd())\\.spinetoolbox\\items\\input_db\\Input DB.sqlite"
using_spinedb(inputDB)

### Sets and indices

$u \in U^{fl}$
: Set of flexible generation units (e.g. single-cycle gas turbine and distillate oil plant).

$u \in U^{in}$
: Set of inflexible generation units (e.g. CCGT and steam-cycle coal generators).

$u \in U^{all}$
: Set of all dispatchable generation units consisting of the flexible and the inflexible, i.e. $U^{all} = U^{fl} \cup U^{in}$.

$v \in V$
: Set of variable renewable (VRE) generation units.

$t \in T$
: Set of time-steps.



In [ ]:
# Implements Sets
U_flex = unit_flex()
U_inflex = unit_inflex()
U_all_dispatch = vcat(U_flex, U_inflex)
V = unit_vre()

t1 = float(start_time(timeframe=timeframe(:solve_6h)))
t_end = float(end_time(timeframe=timeframe(:solve_6h)))
Δt = float(timestep_length(timeframe=timeframe(:solve_6h)))
T = t1:Δt:t_end
T_extend = t1-1.0:Δt:t_end;
# 0.0:float(size(T)[1]), 0.0:float(length(T))

### Variables

$x_{v,t}^{curt}$
: Curtailment (MW) of VRE units.

$x_{u,t}^{gen}$ 
: Electrical output (MW) of dispatchable units.

$z_{u,t:u \in U^{in}}^{online} \in \{0,1\}$ 
: Online decision (*binary*) of inflexible units.

$z_{u,t:u \in U^{in}}^{start} \in \{0,1\}$ 
: Start up decision (*binary*) of inflexible units.

### MIP Formulation - Objective function

$$
\begin{align}
    & \min_{\{ z_{u,t:u \in U^{in}}^{start},z_{u,t:u \in U^{in}}^{online},x_{u,t}^{gen} \}} \sum_{t \in T} C_t^{total} \text{, } \\
    & \textbf{where, } \\
    & C_t^{total} = \sum_{u \in  U^{in}} 
             \Big( C_{u}^{start} \cdot z_{u,t}^{start} + C_{u}^{NL} \cdot z_{u,t}^{online} + C_{u}^{incr} \cdot x_{u,t}^{gen} \Big) 
             + \sum_{u \in  U^{fl}} C_{u}^{avg} \cdot x_{u,t}^{gen} \text{, } \forall t \in T \text{. } \\
\end{align}
$$

In [ ]:
# Set up a model
model_FAST_MIP = Model(Cbc.Optimizer)

# Build variables
@variable(model_FAST_MIP, x_curt[v in V, t in T] >= 0)
@variable(model_FAST_MIP, x_gen[u in U_all_dispatch, t in T] >= 0)
@variable(model_FAST_MIP, z_start[u in U_inflex, t in T], Bin)
@variable(model_FAST_MIP, z_online[u in U_inflex, t in T_extend], Bin)

fix.(z_online[U_inflex, 0.0], 0; force = true)

# Total cost per timestep
@expression(
    model_FAST_MIP, C_total[t in T], 
    sum((cost_start(unit_inflex = u)*z_start[u, t] + cost_NoLoad(unit_inflex = u)*z_online[u, t] + cost_incr(unit_inflex = u)*x_gen[u, t]) for u in U_inflex)
    +
    sum(cost_avg(unit_flex = u)*x_gen[u, t] for u in U_flex)
)

# Objective function
@objective(model_FAST_MIP, Min, sum(C_total[t] for t in T));

### MIP Formulation - Constraints

$$
\begin{align}
    & \sum_{u \in U^{all}} x_{u,t}^{gen} + \sum_{v \in V} \Big( P_{v,t}^{avail} - x_{v,t}^{curt} \Big) = P_t^{dem} \text{, } \forall t \in T \\
    & x_{v,t}^{curt} \leq P_{v,t}^{avail} \text{, } \forall v \in V \text{, } \forall t \in T \\
    & \sum_{u \in U^{in}} \Big( z_{u,t}^{online} \cdot P_{u}^{max} \Big) - \sum_{u \in U^{in}} x_{u,t}^{gen} \geq P_t^{res} \text{, } \forall t \in T \\
    & \sum_{u \in U^{all}} x_{u,t}^{gen} \leq I_{U^{all}} \text{, } \forall t \in T \\
    & \sum_{u \in U^{in}} z_{u,t}^{online} \leq N_{U^{in}} \text{, } \forall t \in T \\
    & x_{u,t}^{gen} \geq z_{u,t}^{online} \cdot P_{u}^{min} \text{, } \forall u \in U^{in} \text{, } \forall t \in T  \\
    & x_{u,t}^{gen} \leq z_{u,t}^{online} \cdot P_{u}^{max} \text{, } \forall u \in U^{in} \text{, } \forall t \in T  \\
    & z_{u,t}^{start} \geq z_{u,t}^{online} -z_{u,t-1}^{online} \text{, } \forall u \in U^{in} \text{, } \forall t \in T.
\end{align}
$$

In [ ]:
# Supply demand balance
@constraint(model_FAST_MIP, supply_demand_balance[t in T], 
    sum(x_gen[u, t] for u in U_all_dispatch) + sum((P_avail(unit_vre=v, inds=t) - x_curt[v,t]) for v in V) 
    == 
    demand(node = node(:Elec), inds=t)
);

In [ ]:
# Curtailment limit
@constraint(model_FAST_MIP, curtail_limit[v in V, t in T], x_curt[v, t] <= P_avail(unit_vre=v, inds=t));

In [ ]:
# Reserve requirement
@constraint(model_FAST_MIP, reserve_requirement[t in T], 
    sum(z_online[u, t]*P_max(unit_inflex=u) for u in U_inflex) - sum(x_gen[u, t] for u in U_inflex) 
    >= 
    min_reserve(node=node(:Elec))
);

In [ ]:
# Dispatch generation limit
I_all_dispatch = sum(installed_capacity(unit_flex = u) for u in U_flex) + sum(P_max(unit_inflex = u) * N(unit_inflex = u) for u in U_inflex)
@constraint(model_FAST_MIP, dispatch_ub[t in T], sum(x_gen[u, t] for u in U_all_dispatch) <= I_all_dispatch);

In [ ]:
# Online unit limit
N_inflex = sum(N(unit_inflex=u) for u in U_inflex)
@constraint(model_FAST_MIP, online_ub[t in T], sum(z_online[u, t] for u in U_inflex) <= N_inflex);

In [ ]:
# Inflex generation lower bound
@constraint(model_FAST_MIP, inflex_lb[u in U_inflex, t in T], x_gen[u, t] >= z_online[u, t]*P_min(unit_inflex = u));

In [ ]:
# Inflex generation upper bound
@constraint(model_FAST_MIP, inflex_ub[u in U_inflex, t in T], x_gen[u, t] <= z_online[u, t]*P_max(unit_inflex = u));

In [ ]:
# Inflex unit startup transition
@constraint(model_FAST_MIP, inflex_su[u in U_inflex, t in T], z_start[u, t] >= z_online[u, t] - z_online[u, t-1.0]);

### Solve MIP

In [ ]:
optimizer = HiGHS.Optimizer 
# HiGHS.Optimizer, Cbc.Optimizer
set_optimizer(model_FAST_MIP, optimizer; add_bridges = false)
set_silent(model_FAST_MIP)
# unset_silent(model_FAST_MIP)

optimize!(model_FAST_MIP)                          # Solve the problem              

println("Model status = $(termination_status(model_FAST_MIP))") # Solution status

println("Solve time = $(solve_time(model_FAST_MIP))")

println("Objective value = $(objective_value(model_FAST_MIP))")

# @show solution_summary(model_FAST_MIP)

# @show value.(x_gen)

# @show value.(x_curt)

# @show value.(z_start)

# @show value.(z_online);

### Solve LP with integer variable fixed to obtain duals

In [ ]:
# Copy the MIP model
model_FAST_LP, map = copy_model(model_FAST_MIP)

# Fix the binary variables solved in the MIP
z_online_fix = map[z_online]
z_start_fix = map[z_start]

fix.(z_online_fix[U_inflex, T_extend], value.(z_online[U_inflex, T_extend]); force = true)
fix.(z_start_fix[U_inflex, T], value.(z_start[U_inflex, T]); force = true)

# Assign a LP solver
optimizer = Clp.Optimizer 
set_optimizer(model_FAST_LP, optimizer; add_bridges = false)
set_silent(model_FAST_LP)
# unset_silent(model_FAST_LP)

optimize!(model_FAST_LP)                          # Solve the problem              

println("Model status = $(termination_status(model_FAST_LP))") # Solution status
println("Solve time = $(solve_time(model_FAST_LP))")

println("Objective value = $(objective_value(model_FAST_LP))")
println("Has duals = $(has_duals(model_FAST_LP))")
if has_duals(model_FAST_LP)
    println("Dual status = $(dual_status(model_FAST_LP))")

    println("Shadow price of demand = $(shadow_price.(map[supply_demand_balance]))")

    println("Reduced cost of generation = $(reduced_cost.(map[x_gen]))")
end

# @show solution_summary(model_FAST_LP)

# @show value.(map[x_gen])

# @show value.(map[x_curt])

# @show value.(z_start_fix)

# @show value.(z_online_fix);

In [ ]:
@show typeof(supply_demand_balance[1.0])
typeof(x_gen[unit_inflex(:ST_Coal), 1.0])
typeof(x_gen[U_all_dispatch[1], 1.0])
value.(x_gen[U_all_dispatch, 1.0])
value(x_gen[U_all_dispatch[2], 1.0])
value(x_gen[unit_flex(:OCGT), 3.0])

object_dictionary(model_FAST_LP)
model_FAST_LP[:z_online] == map[z_online] == z_online_fix != z_online == model_FAST_MIP[:z_online]

In [ ]:
model_FAST_MIP
# Check elements of the model
object_dictionary(model_FAST_MIP)
model_FAST_MIP[:supply_demand_balance]
@show objective_function(model_FAST_MIP)